In [1]:
from psycopg2 import connect as cnct
from statsmodels.regression import linear_model
from sklearn.metrics import mean_squared_error as mse
from scipy.optimize import minimize, basinhopping, Bounds, LinearConstraint

import datetime
import statsmodels.api as sm
import copy
import os
import numpy as np
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import pyflux as pf
import seaborn

# matplotlib.use("pgf")
# matplotlib.rcParams.update({
#     "pgf.texsystem": "pdflatex",
#     'font.family': 'serif',
#     'text.usetex': True,
#     'pgf.rcfonts': False,
# })

# 0. Глобальные настройки

In [2]:
DB_SETTINGS = {
    'dbname'   : 'orderlogs', 
    'port'     :  5432, 
    'user'     : 'postgres', 
    'host'     : 'localhost',
    'password' : ''
}

ENGINE_SETTINGS = 'postgresql://postgres@localhost:5432/orderlogs'

with cnct(**DB_SETTINGS) as conn:
    cur = conn.cursor()
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
    TABLE_NAMES = cur.fetchall()
    TABLE_NAMES = sorted(list(map(lambda x: x[0], TABLE_NAMES)))

TABLE_NAMES_ORDERLOGS = list(filter(lambda x: len(x) == 8, TABLE_NAMES))
TABLE_NAMES_CHAINS = list(filter(lambda x: len(x) > 8, TABLE_NAMES))
TICKERS = list(set(map(lambda x: x[9:], TABLE_NAMES_CHAINS)))
PATH = os.path.abspath('')

# 1. Проблема ликвидации

## 1.1. Реализация

### 1.1.0. Получение оценок параметров

#### 1.1.0.1. Получение оценок для функции издержек

In [3]:
class LOB:
    '''This class replicates the logic for the limit order book building
    '''
    def __init__(self, ticker, date=None, time=None):
        self._ticker = ticker
        self._date = date
        self._time = time
        
        self._lob = None
    
    def table_build(self, date=None, time=None):
        '''This method builds the limit order book table on the specified date and time
        
        :date: the date in int format (for example, 20190603)
        :time: the time in int format (for example, 121059123456)
        :return: the pandas.DataFrame object with columns BUYSELL, ORDERNO, PRICE, VOLUME
        '''
        if date is None and time is None:
            date = self._date
            time = self._time
        elif date is None or time is None:
            raise ValueError('You must specify both date and time')
            
        with cnct(**DB_SETTINGS) as conn:
            cur = conn.cursor()

            table_name = date
            seccode = self._ticker

            query = '''\
            SELECT 
                tt."BUYSELL", tt."PRICE", sum("VOLUME") as "VOLUME"
            FROM
                (SELECT 
                    t."BUYSELL", t."ORDERNO", max(t."PRICE") as "PRICE", sum("VOLUME") as "VOLUME"
                FROM 
                    (SELECT
                        "BUYSELL", "ORDERNO", "ACTION", max("PRICE") as "PRICE",
                        CASE "ACTION" WHEN 1 THEN sum("VOLUME")
                                      WHEN 2 THEN sum(-"VOLUME")
                                      WHEN 0 THEN sum(-"VOLUME")
                        END as "VOLUME"
                    FROM 
                        "{}"
                    WHERE
                        "SECCODE" = '{}' AND "TIME" <= {} AND "PRICE" != 0
                    GROUP BY
                        "BUYSELL", 
                        "ORDERNO",
                        "ACTION") t
                GROUP BY
                    "BUYSELL",
                    "ORDERNO"
                HAVING
                    sum("VOLUME") > 0) tt
            GROUP BY
                "BUYSELL",
                "PRICE"
            ORDER BY
                "BUYSELL" DESC, 
                "PRICE" DESC;
            '''.format(table_name, seccode, time)
            query = " ".join(query.split())

            cur.execute(query)

            limit_order_book = pd.DataFrame(cur.fetchall(), 
                                            columns=['BUYSELL', 'PRICE', 'VOLUME'])
        self._lob = limit_order_book    
        
        return limit_order_book
    
    def lob_plot(self, date, time, buylevels=20, selllevels=20, path=None):
        '''This method builds the limit order book table and plot it on the specified date and time
        
        :date: the date in int format (for example, 20190603)
        :time: the time in int format (for example, 121059123456)
        :buylevels: the number of price levels of BID side, int
        :selllevels: the number of price levels of ASK side, int
        :path: whether you want to save the plot, set the path
        :return: None, matplotlib.pyplot hist returns
        '''
        
        lob = self.table_build(date, time)
        buy = (lob.loc[lob['BUYSELL'] == 'B', ['PRICE', 'VOLUME']]).reset_index(drop=True)
        sell = (lob.loc[lob['BUYSELL'] == 'S', ['PRICE', 'VOLUME']]).reset_index(drop=True)
        
        # code to make beautiful barplot
        bid = np.max(buy.PRICE)
        ask = np.min(sell.PRICE)
        plt.figure()
        plt.barh(sell.PRICE[-selllevels:].reset_index(drop=True), 
                 sell.VOLUME[-selllevels:].reset_index(drop=True), 
                 height=0.01, label = 'SELL', color = 'r')
        plt.barh(buy.PRICE[:buylevels].reset_index(drop=True), 
                 -buy.VOLUME[:buylevels].reset_index(drop=True),
                 height=0.01, label = 'BUY', color = 'g')
        plt.xticks(rotation=15)

        plt.xlabel('VOLUME')
        plt.ylabel('PRICE')
        plt.title('LOB for ' + self._ticker +\
                  ' at ' + str(date)[:4] + '-' + str(date)[4:6] + '-' + str(date)[6:] +\
                  ': ' +\
                  str(time)[:2] + ':' + str(time)[2:4] + ':' + str(time)[4:6] + '.' + str(time)[6:])
        plt.legend(['ASK = ' + str(ask), 
                    'BID = ' + str(bid)])
        plt.grid()
        
        if path is None:
            plt.show()
        else:
            plt.savefig(os.path.join(os.path.abspath(''), path), bbox_inches='tight')
            plt.close()
    
    def price_calculate(self):
        ask = np.min(self._lob.loc[self._lob['BUYSELL'] == 'S', 'PRICE'])
        bid = np.max(self._lob.loc[self._lob['BUYSELL'] == 'B', 'PRICE'])
        
        return (ask + bid) / 2

In [4]:
class Perold(LOB):
    '''This class implements the logic for Perold transaction costs calculation
    '''
    def __init__(self, ticker, date, time):
        '''
        :ticker: the string of fin. instrument's name ('SBER', for example)
        :date: the date in int (or str) format (for example, 20190603)
        :time: the time in int format (for example, 111050000000)
        '''
        super().__init__(ticker)
        self._date = date
        self._time = time
        self._lob = self.table_build(date, time)
        self._bid = np.max(self._lob.loc[self._lob['BUYSELL'] == 'B', 'PRICE'])
        self._ask = np.min(self._lob.loc[self._lob['BUYSELL'] == 'S', 'PRICE'])
        self._halfbidask = (self._bid + self._ask) / 2
        
        self._tr_costs = None

    def empirical_cost_function_build(self):
        '''This method builds the pd.DataFrame object that consists the following fields:
        VOLUME, COSTS according to the Perold's function θ(·)= Σ(p_i - p)V_i
        '''
        lob = self._lob
        lob['VOLUME'] = lob['VOLUME'].astype(int)
        buy = lob.loc[lob['BUYSELL'] == 'B', ['PRICE', 'VOLUME']].sort_values(by='PRICE', ascending=False).reset_index(drop=True)
        
        tr_costs_sell = copy.deepcopy(buy)
        tr_costs_sell['COSTS'] = np.cumsum(np.abs(tr_costs_sell['PRICE'] - self._halfbidask) * tr_costs_sell['VOLUME'])
        tr_costs_sell['VOLUME'] = np.cumsum(tr_costs_sell['VOLUME'])
        tr_costs_sell = tr_costs_sell[['VOLUME', 'COSTS']]
        
        sell = lob.loc[lob['BUYSELL'] == 'S', ['PRICE', 'VOLUME']].sort_values(by='PRICE', ascending=True).reset_index(drop=True)

        tr_costs_buy = copy.deepcopy(sell)
        tr_costs_buy['COSTS'] = np.cumsum(np.abs(tr_costs_buy['PRICE'] - self._halfbidask) * tr_costs_buy['VOLUME'])
        tr_costs_buy['VOLUME'] = np.cumsum(tr_costs_buy['VOLUME'])
        tr_costs_buy = tr_costs_buy[['VOLUME', 'COSTS']]
        tr_costs_buy['VOLUME'] = -tr_costs_buy['VOLUME']
        
        tr_costs_all = pd.concat((tr_costs_buy, tr_costs_sell)).sort_values(by='VOLUME').reset_index(drop=True)
        
        
        self._tr_costs = tr_costs_all

        return self._tr_costs

    def plot_empirical_function(self, path=None):
        '''This method plots the empirical function of costs and saves it into the given path (if None, 
        it will not be saved)
        :path: the string value (relative path where to save the plot), for example (./graph.pdf)
        '''
        if self._tr_costs is None:
            print('Just build the cost table at once')
        else:
            plt.figure()
            plt.plot(self._tr_costs['VOLUME'], self._tr_costs['COSTS'])
            plt.xticks(rotation=15)
            plt.title('Empirical Function of Transaction Costs')
            plt.xlabel('VOLUME, positive for sell, negative for buy')
            plt.ylabel('TRANSACTION COSTS')
            if path is None:
                plt.show() 
            else:
                plt.savefig(os.path.join(os.path.abspath(''), path), bbox_inches='tight')
                plt.close()

In [5]:
class Liquidity(Perold):
    '''This class implements the logic for α and β estimation
    '''
    def __init__(self, ticker, date, time):
        super().__init__(ticker, date, time)
        self.empirical_cost_function_build()
    
    def calculate_costs_function_parameters(self, method='squared'):
        '''This method implements the logic for the α and β estimation
        
        :param: method is the linear model for transaction costs:
            squared: volume, volume squared
        '''
        V = np.array(self._tr_costs.VOLUME)
        costs = np.array(self._tr_costs.COSTS)
        
        if method == 'squared':
            y = costs
            X = np.stack([V, V**2], axis=1)
            
            model = linear_model.OLS(y, X)
            results = model.fit()
            
            return results.params, results.cov_params()

#### 1.1.0.2. Получение оценок для волатильности

Составление необходимых временных точек

In [6]:
def time_convertion(time: int, date=20190603) -> datetime.datetime:
    s_time = str(time)
    s_date = str(date)
    
    hours = int(s_time[:2])
    minutes = int(s_time[2:4])
    seconds = int(s_time[4:6])
    microseconds = int(s_time[6:])
    
    year = int(s_date[:4])
    month = int(s_date[4:6])
    day = int(s_date[6:])
    
    time_datetime = datetime.datetime(year=year, month=month, day=day, 
                                      hour=hours, minute=minutes, second=seconds, 
                                      microsecond=microseconds)
    
    return time_datetime

def time_convertion_back(time):
    hour = time.hour
    minute = time.minute
    second = time.second
    microsecond = time.microsecond
    
    result = ('{:02}'.format(hour) + 
              '{:02}'.format(minute) + 
              '{:02}'.format(second) + 
              '{:06}'.format(microsecond))
    
    return int(result)

In [7]:
def time_creation(start: int, end: int, delta=5) -> list:
    start = time_convertion(start)
    end = time_convertion(end)
    new_value = start
    result = []
    while new_value <= end:
        result.append(new_value)
        new_value += datetime.timedelta(seconds=delta)
        
    result = list(map(lambda x: time_convertion_back(x), result))
    
    return result

Получение временных рядов

In [8]:
def price_series(start, end, ticker='SBER', date=20190603, delta=5):
    times = time_creation(start, end, delta=delta)
    prices = []
    for time in times:
        print(time)
        lob = LOB(ticker=ticker, date=date, time=time)
        lob.table_build()
        prices.append(lob.price_calculate())
    
    return pd.DataFrame(np.array(list(zip(times, prices))), columns=['Time', 'Price'])

Получение прогнозов параметров волатильности

In [9]:
def vola_forecast(*, filename, time_multiplier, number_iterations):
    '''This function calculates the prediction of vola through N steps using the filaname.csv with the 
    time series of fin. instrument's prices. 
    
    :param filename: str (the name of file that consists of time series of prices)
    :param time_multiplier: if the volatility prediction is every 5 seconds. And the liquidation occurs 
    every minute then time_multiplier is 12
    :param number_iterations: the number of liquidation iterations
    '''
    path_to_filename = os.path.join(PATH, 'time_series_for_volatility/')
    path_to_filename = os.path.join(path_to_filename, filename)
    df = pd.read_csv(path_to_filename, index_col=0)

    price = df.Price

    dprice = np.log(price) - np.log(price.shift(1))
    
    dprice = pd.DataFrame(dprice.dropna().reset_index(drop=True))
    
    model = pf.GARCH(dprice, p=1, q=1)
    model.fit()
    
    forecast_initial = model.predict(h=number_iterations*time_multiplier)
    
    j = 0
    forecast_for_liquidation = []
    while j < len(forecast_initial):
        forecast_for_liquidation.append(np.sum(forecast_initial[j:j+time_multiplier]))
        j += time_multiplier
    
    forecast_for_liquidation = np.array(forecast_for_liquidation)
    
    return forecast_for_liquidation

### 1.1.1. Решение оптимизационной задачи методом доверительных областей

In [10]:
class Optimizer:
    '''This class solves the optimization problem according to the master's thesis
    '''
    def __init__(self, *, lambda_parameter, volume_to_liquidate, number_iterations):
        self._lambda = lambda_parameter
        self._volume = volume_to_liquidate
        self._N = number_iterations
            
    @staticmethod    
    def _expected_costs(v, params):
        return params[0] * v + params[1] * v**2
    
    @staticmethod
    def _expected_costs_d(v, params):
        return params[0] + 2 * params[1] * v
    
    @staticmethod
    def _expected_costs_dd(v, params):
        return 2 * params[1]
    
    @staticmethod
    def _var_costs(v, params, cov_params):
        var_b1 = cov_params[0, 0]
        var_b2 = cov_params[1, 1]
        cov_b1b2 = cov_params[0, 1]

        var = v**2 * var_b1 + v**4 * var_b2 + 2 * cov_b1b2 * v**3
        return var
    
    @staticmethod
    def _var_costs_d(v, params, cov_params):
        var_b1 = cov_params[0, 0]
        var_b2 = cov_params[1, 1]
        cov_b1b2 = cov_params[0, 1]
    
        var = 2 * v * var_b1 + 4 * v**3 * var_b2 + 6 * cov_b1b2 * v**2
        return var
    
    @staticmethod
    def _var_costs_dd(v, params, cov_params):
        var_b1 = cov_params[0, 0]
        var_b2 = cov_params[1, 1]
        cov_b1b2 = cov_params[0, 1]
    
        var = 2 * var_b1 + 12 * v**2 * var_b2 + 12 * cov_b1b2 * v
        return var
    
    def _solve(self, *, sigmas_sq, params_costs, cov_params_costs, x0=None):
        if len(sigmas_sq) != self._N:
            raise ValueError('You must specify the vector of parameters ' + 
                             'for sigma of the length of number iterations')
        if x0 is None:
            x0 = np.array([self._volume / self._N] * self._N)
        bounds = Bounds([0] * self._N, [np.inf] * self._N)
        linear_constraint = LinearConstraint([1] * self._N, self._volume, self._volume)
        
        V = self._volume
        lam = self._lambda
        s2 = sigmas_sq
        def functional(v):

            v2_new = np.cumsum(v[::-1])[::-1] ** 2 
            square = np.sqrt(np.sum(s2 * v2_new) + 
                             np.sum(self._var_costs(v, params_costs, cov_params_costs)))
            first = lam * square
            second = np.sum(self._expected_costs(v, params_costs))
            func = first + second

            return func

        def functional_jac(v):

            coeff = lam / 2
            v_new = np.cumsum(v[::-1])[::-1]
            v2_new = v_new  ** 2 
            denumerator = np.sqrt(np.sum(s2 * v2_new) + 
                                  np.sum(self._var_costs(v, params_costs, cov_params_costs)))

            jac = np.empty(shape=len(v))
            for i in range(len(v)):
                v_i = v[i]
                sl = (s2 * v_new)[:i+1]
                numerator_i = (self._var_costs_dd(v_i, params_costs, cov_params_costs) +
                               2 * np.sum(sl))         
                additional_i = self._expected_costs_d(v_i, params_costs)
                jac_i = coeff * numerator_i / denumerator + additional_i
                jac[i] = jac_i

            return jac

        def functional_hess(v):
            coeff = lam / 2
            v_new = np.cumsum(v[::-1])[::-1]
            v2_new = v_new**2
            g = (np.sqrt(np.sum(s2 * v2_new) + 
                         np.sum(self._var_costs(v, params_costs,cov_params_costs))))
            hess = np.empty(shape=(len(v), len(v)))
            for i in range(len(v)):
                for j in range(len(v)):

                    f_i = self._var_costs_d(v[i], params_costs, cov_params_costs) + 2 * np.sum((s2 * v_new)[:i])
                    f_ij = (self._var_costs_dd(v[j], params_costs, cov_params_costs) * (i == j) + 
                            2 * np.sum(s2[:np.min([i, j])]))

                    g_j = (0.5 * (self._var_costs_d(v[j], params_costs, cov_params_costs) + 2 * np.sum((s2 * v_new)[:j])) / 
                           np.sqrt(np.sum(s2 * v2_new) + np.sum(self._var_costs(v, params_costs, cov_params_costs))))

                    brackets = f_ij * g - g_j * f_i
                    hess_ij = (coeff * 
                               (brackets) / 
                               g**2 + 
                               self._expected_costs_dd(v, params_costs) * (i == j))

                    hess[i, j] = hess_ij

            return hess
        
        minimizer_kwargs = {'method':'trust-constr', 'jac':functional_jac, 'hess':functional_hess,
                            'constraints':[linear_constraint], 'bounds':bounds}
        res = basinhopping(functional, x0, minimizer_kwargs=minimizer_kwargs)
        
        return res
    
    def solve(self, *, sigmas_sq, params_costs, cov_params_costs, x0=None, return_left=False):
        result = self._solve(sigmas_sq=sigmas_sq, 
                             params_costs=params_costs, 
                             cov_params_costs=cov_params_costs, 
                             x0=x0)
        
        strategy = result.x 
        left_position = self._volume - np.cumsum(strategy)
        left_position = np.concatenate([np.array([self._volume]), left_position])
        
        if return_left:
            return left_position
        return strategy

# 2. Решение для всех поставленных задач

In [11]:
def problem_solution(*, ticker, date_of_liquidation, time_of_liquidation, 
                     number_of_iteration, duration_of_iteration, lambda_param, volume,
                     hour_interval=1, delta_prices=5, return_left=False):
    '''This function solves the optimal liquidation problem at all
    
    :param ticker: str value of ticker (for example, SBER)
    :param date_of_liquidation: int value of the date of liquidation (for example, 20190603)
    :param time_of_liquidation: int value of the time of liquidation (for example, 115023000000)
    :param number_of_iteration: int value of number of iterations of liquidation strategy (for example, 15)
    :param duration_of_iteration: int value of duration in seconds that each liquidation step occurs (for example,
        60 means that each 1 minute)
    :param lambda_param: lambda parameter of optimization problem
    :param volume: int value of sum volume that is needed to liquidate (for example, 10000)
    :param hour_interval: int value of backward time interval for prices calculation that are need for 
        volatility estimation (for example, 1 means that the interval is 105023000000 - 115023000000)
    :param delta_prices: the interval of price calculation in seconds (for example, 5 means each 5 seconds)
    :param return_left: boolean parameter (False - return strategy of liquidation, 
        True - return the left position after each step of liquidation)
    '''
    liq = Liquidity(ticker=ticker, date=date_of_liquidation, time=time_of_liquidation)
    params, cov_params = liq.calculate_costs_function_parameters()
    
    path_to_prices = os.path.join(PATH, 'time_series_for_volatility')
    filename = (ticker + '_' + str(date_of_liquidation) + '_' + str(time_of_liquidation) + '_' + 
                str(hour_interval) + '_' + str(delta_prices) + '.csv')
    
    if not os.path.isfile(os.path.join(path_to_prices, filename)):
        
        time_series_last = None
        listdir = list(filter(lambda x: x[-3:] == 'csv', 
                              os.listdir(path_to_prices)))
        if listdir:
            listdir = list(map(lambda x: x.split('_'), listdir))
            listdir = list(filter(lambda x: x[0] == ticker and x[1] == str(date_of_liquidation) and 
                                  x[3] == str(hour_interval) and x[4][:-4] == str(delta_prices), 
                                  listdir))
            if listdir:
                last_time = max(list(map(lambda x: x[2], listdir)))
                file_last = list(filter(lambda x: x[2] == last_time, listdir))[0]
                file_last = '_'.join(file_last)
                
                time_series_last = pd.read_csv(os.path.join(path_to_prices, file_last), index_col=0)

        if time_series_last is None:
            time_series = price_series(time_of_liquidation-hour_interval*10**10, 
                                       time_of_liquidation, delta=delta_prices, 
                                       date=date_of_liquidation)
        else:
            start = time_convertion(int(last_time)) + datetime.timedelta(seconds=delta_prices)
            start = time_convertion_back(start)
            time_series_new = price_series(int(start), 
                                           time_of_liquidation, delta=delta_prices, 
                                           date=date_of_liquidation)
            time_series = pd.concat((time_series_last, time_series_new), axis=0).reset_index(drop=True)
            length = len(time_series_last)
            time_series = (time_series.iloc[-length:, :]).reset_index(drop=True)

        time_series.to_csv(os.path.join(path_to_prices, filename))
        
    sigmas_sq_to_opt = vola_forecast(filename=filename, 
                                     time_multiplier=duration_of_iteration//delta_prices, 
                                     number_iterations=number_of_iteration)
        
    opt = Optimizer(lambda_parameter=lambda_param, 
                    volume_to_liquidate=volume, 
                    number_iterations=number_of_iteration)
    
    strategy = opt.solve(sigmas_sq=sigmas_sq_to_opt, 
                         params_costs=params, 
                         cov_params_costs=cov_params,
                         return_left=return_left)
    
    return strategy

In [12]:
# np.random.seed(1234)
# lams = [0.1, 1, 5, 100]

# sols = []
# for lam in lams:
#     print(lam)
#     PARAMS_TASK = {
#         'ticker': 'SBER', 
#         'date_of_liquidation': 20190603, 
#         'time_of_liquidation': 114000000000, 
#         'number_of_iteration': 15, 
#         'duration_of_iteration': 60, 
#         'lambda_param': lam, 
#         'volume': 10000,
#         'return_left': True
#     }
#     sol = problem_solution(**PARAMS_TASK), lam
#     print(np.round(sol[0]))
    
#     sols.append(sol)

In [13]:
# figure, ax = plt.subplots(1, 1, sharey=False)
# 
# colors = ['#000000', '#606060', '#888888', '#D3D3D3']
# for j in range(len(sols)):
#     ax.plot(sols[j][0], color=colors[j])
# 
# ax.set_ylabel('Volume')
# ax.set_xlabel('Number of iteration')
# 
# ax.legend([r'$\lambda = {}$'.format(x) for x in list(map(lambda x: x[1], sols))])
# figure.set_size_inches(w=6, h=4)
# 
# plt.savefig(os.path.join(PATH, 'figures_to_report/optimization_demo.pgf'), 
#             bbox_inches='tight')

In [14]:
# np.random.seed(1234)
# iters = [3, 5, 7, 15]
# sols = []
# for iter_ in iters:
#     print(iter_)
#     PARAMS_TASK = {
#         'ticker': 'SBER',
#         'date_of_liquidation': 20190603,
#         'time_of_liquidation': 114000000000,
#         'number_of_iteration': iter_,
#         'duration_of_iteration': 60,
#         'lambda_param': 5,
#         'volume': 10000,
#         'return_left': True
#     }
#     sol = problem_solution(**PARAMS_TASK), iter_
#     print(np.round(sol[0]))
    
#     sols.append(sol)

In [15]:
# figure, ax = plt.subplots(1, 1, sharey=False)

# colors = ['#000000', '#303030', '#888888', '#D3D3D3']
# for j in range(len(sols)):
#     ax.plot(sols[j][0], color=colors[j])

# ax.set_ylabel('Volume')
# ax.set_xlabel('Number of iteration')

# ax.legend([r'$N = {}$'.format(x) for x in list(map(lambda x: x[1], sols))])
# figure.set_size_inches(w=6, h=4)

# plt.savefig(os.path.join(PATH, 'figures_to_report/optimization_demo_iters.pgf'), 
#             bbox_inches='tight')

# 3. Проверка "оптимальности" ликвидации

## 3.1. Получение средних значений

In [16]:
def count_all_volume(*, ticker):
    result_all = {
        'date': [],
        'ticker': [],
        'volume': []
    }

    for table_name in TABLE_NAMES_ORDERLOGS:
        with cnct(**DB_SETTINGS) as conn:
            cur = conn.cursor()
            query = '''
            SELECT SUM("VOLUME") FROM "{}" WHERE
            "ACTION" = 2 and "SECCODE" = '{}';
            '''.format(table_name, ticker)
            query = ' '.join(query.split())
            cur.execute(query)
            s = int(cur.fetchone()[0])

        result_one = {
            'date': int(table_name), 
            'ticker': ticker, 
            'volume': s
        }

        for key in result_one.keys():
            result_all[key].append(result_one[key])
                
    return result_all

In [17]:
def all_volumes_by_day(*, ticker):
    path = os.path.join(PATH, 
                        'volumes_each_day/{}_traded_volumes.csv'.format(ticker))
    if not os.path.isfile(path):
        data = pd.DataFrame.from_dict(count_all_volume(ticker=ticker))
        data.to_csv(path)        

        return data
    else:
        data = pd.read_csv(path, index_col=0)
        
        return data   

## 3.2. Подстановка и графики

In [18]:
class LiquidationValue:
    '''This class implements the logic for backward estimation of liquidation value of 
    liquidation strategy
    '''
    def __init__(self, *, optimal_strategy, 
                 ticker, date_of_liquidation, time_of_liquidation, 
                 duration_of_iteration=60):
        '''Init the object to calculate liquidation value
        
        :param optimal_strategy: np.array that represents the liquidation strategy 
            (left_position variant)
        :param ticker: the ticker of liquidation
        :param date_of_liquidation: the date of liquidation
        :param time_of_liquidation: the time of liquidation
        :duration_of_iteration: the duration of each step of liquidation in seconds
        '''
        self._ticker = ticker
        self._date = date_of_liquidation
        self._time = time_of_liquidation
        self._duration = duration_of_iteration
        
        self._optimal = np.round(optimal_strategy)
        self._volume = self._optimal[0]
        self._N = len(self._optimal) - 1
        self._optimal = self._optimal[:-1] - self._optimal[1:]
        self._optimal = self._optimal.astype(int)
        
        self._equal = self._volume - np.cumsum(np.array([self._volume / 
                                                         self._N] * self._N))
        self._equal = np.concatenate([np.array([self._volume]), self._equal])
        self._equal = np.round(self._equal)
        self._equal = self._equal[:-1] - self._equal[1:]
        self._equal = self._equal.astype(int)
        
        self._first = np.concatenate([np.array([self._volume]), 
                                      np.array([0] * (self._N - 1))])
        
        self._times = None
        self._lobs = None
        self._initial_price = None
        
    
    def _calculate_times(self):
        if self._times is None:
            start_time = time_convertion(self._time) + datetime.timedelta(seconds=self._duration)
            end_time = start_time + datetime.timedelta(seconds=(self._N-1)*self._duration)

            start_time = time_convertion_back(start_time)
            end_time = time_convertion_back(end_time)

            times = time_creation(start_time, end_time, delta=self._duration)
            
            self._times = times
            
    def _calculate_lobs(self):
        if self._lobs is None:
            self._calculate_times()
            
            self._lobs = []
            for time in self._times:
                lob = LOB(ticker=self._ticker, date=self._date, time=time)
                lob.table_build()
                if time == self._times[0]:
                    self._initial_price = (np.max(lob._lob.loc[lob._lob['BUYSELL'] == 'B', 'PRICE']) +
                                           np.min(lob._lob.loc[lob._lob['BUYSELL'] == 'S', 'PRICE'])) / 2
                table = lob._lob.loc[lob._lob['BUYSELL'] == 'B', ['PRICE', 
                                                                  'VOLUME']].reset_index(drop=True)
                
                table['PRICE'] = table['PRICE'].astype(float)
                table['VOLUME'] = table['VOLUME'].astype(int)
                
                self._lobs.append(table)    
    
    def _calculate_value(self, strategy='optimal'):
        '''This method calculates the liquidation value according to the given strategy
        
        :param strategy: str ('optimal', 'equal' of 'first')
        '''
        if strategy == 'optimal':
            strategy = self._optimal
        elif strategy == 'equal':
            strategy = self._equal
        elif strategy == 'first':
            strategy = self._first
        else:
            raise(ValueError('You must specify the type of strategy correctly'))
            
        self._calculate_lobs()
        
        liq_values = []
        try:
            for step in range(len(strategy)):
                liquidate_volume_onestep = strategy[step]

                if liquidate_volume_onestep <= int(self._lobs[step].VOLUME[0]):
                    liq_value = liquidate_volume_onestep * float(self._lobs[step].PRICE[0])

                else:
                    table = copy.deepcopy(self._lobs[step])
                    table['VOLUME_CUM'] = np.cumsum(table['VOLUME'])
                    table_less = table.loc[table['VOLUME_CUM'] < liquidate_volume_onestep]
                    liq_value = np.sum(table_less.VOLUME * table_less.PRICE)

                    price_more = (table.loc[table['VOLUME_CUM'] >= liquidate_volume_onestep, 'PRICE'].reset_index(drop=True))[0]
                    left_volume = liquidate_volume_onestep - np.sum(table_less['VOLUME'])
                    liq_value += price_more * left_volume

                liq_values.append(liq_value)
        except:
            liq_values = np.nan
        
        return np.sum(liq_values)
    
    def calculate_liquidation_value(self):
        liquidation_values = {
            'time': self._time,
            'date': self._date
        }
        for strategy_type in ['optimal', 'equal', 'first']:
            liquidation_values[strategy_type] = self._calculate_value(strategy=strategy_type)
        
        return liquidation_values

In [19]:
data_volume = all_volumes_by_day(ticker='SBER')
data_volume

,date,ticker,volume
0,20190603,SBER,164844820
1,20190604,SBER,137309220
2,20190605,SBER,103273460
3,20190606,SBER,119707520
4,20190607,SBER,133357000
5,20190610,SBER,121256300
6,20190611,SBER,131358700
7,20190613,SBER,146484500
8,20190614,SBER,98305600
9,20190617,SBER,64373160


In [20]:
np.mean(data_volume.volume)

103529670.5263158

In [1]:
164844820 / (8.75 * (6))

3139901.3333333335

In [21]:
np.random.seed(1234)
result = {
    'date': [],
    'time': [],
    'optimal': [],
    'equal': [],
    'first': []
}
number_iters = 20
duration_iters = 30
for date in [20190624, 20190603]:
    if date == 20190603:
        volume_paste = int(np.round(164844820 / (8.75 * (6)), 0))
    elif date == 20190624:
        volume_paste = int(np.round(53628360 / (8.75 * (6)), 0))
    else:
        raise(ValueError('Something went wrong'))
    
    
    
    for time in time_creation(start=110000000000, 
                              end=130000000000, 
                              delta=5*60):

        PARAMS_TASK = {
            'ticker': 'SBER',
            'date_of_liquidation': date,
            'time_of_liquidation': time,
            'number_of_iteration': number_iters,
            'duration_of_iteration': duration_iters,
            'lambda_param': 1,
            'volume': volume_paste,
            'return_left': True
        }
        sol = problem_solution(**PARAMS_TASK)

        strategy = sol
        
        liq_value = LiquidationValue(optimal_strategy=strategy,
                                     ticker='SBER',
                                     date_of_liquidation=date,
                                     time_of_liquidation=time, 
                                     duration_of_iteration=duration_iters)

        value = liq_value.calculate_liquidation_value()

        for key in value.keys():
            result[key].append(value[key])
        
        print(pd.concat([pd.DataFrame.from_dict(result), (pd.DataFrame.from_dict(result)['optimal'] -
                                                          pd.DataFrame.from_dict(result)['equal']) ], 
                        axis=1))
    

data_diff = pd.DataFrame.from_dict(result)

100000000000
100005000000
100010000000
100015000000
100020000000
100025000000
100030000000
100035000000
100040000000
100045000000
100050000000
100055000000
100100000000
100105000000
100110000000
100115000000
100120000000
100125000000
100130000000
100135000000
100140000000
100145000000
100150000000
100155000000
100200000000
100205000000
100210000000
100215000000
100220000000
100225000000
100230000000
100235000000
100240000000
100245000000
100250000000
100255000000
100300000000
100305000000
100310000000
100315000000
100320000000
100325000000
100330000000
100335000000
100340000000
100345000000
100350000000
100355000000
100400000000
100405000000
100410000000
100415000000
100420000000
100425000000
100430000000
100435000000
100440000000
100445000000
100450000000
100455000000
100500000000
100505000000
100510000000
100515000000
100520000000
100525000000
100530000000
100535000000
100540000000
100545000000
100550000000
100555000000
100600000000
100605000000
100610000000
100615000000
100620000000

105235000000
105240000000
105245000000
105250000000
105255000000
105300000000
105305000000
105310000000
105315000000
105320000000
105325000000
105330000000
105335000000
105340000000
105345000000
105350000000
105355000000
105400000000
105405000000
105410000000
105415000000
105420000000
105425000000
105430000000
105435000000
105440000000
105445000000
105450000000
105455000000
105500000000
105505000000
105510000000
105515000000
105520000000
105525000000
105530000000
105535000000
105540000000
105545000000
105550000000
105555000000
105600000000
105605000000
105610000000
105615000000
105620000000
105625000000
105630000000
105635000000
105640000000
105645000000
105650000000
105655000000
105700000000
105705000000
105710000000
105715000000
105720000000
105725000000
105730000000
105735000000
105740000000
105745000000
105750000000
105755000000
105800000000
105805000000
105810000000
105815000000
105820000000
105825000000
105830000000
105835000000
105840000000
105845000000
105850000000
105855000000

['SBER', '20190624', '113000000000', '1', '5.csv']
113005000000
113010000000
113015000000
113020000000
113025000000
113030000000
113035000000
113040000000
113045000000
113050000000
113055000000
113100000000
113105000000
113110000000
113115000000
113120000000
113125000000
113130000000
113135000000
113140000000
113145000000
113150000000
113155000000
113200000000
113205000000
113210000000
113215000000
113220000000
113225000000
113230000000
113235000000
113240000000
113245000000
113250000000
113255000000
113300000000
113305000000
113310000000
113315000000
113320000000
113325000000
113330000000
113335000000
113340000000
113345000000
113350000000
113355000000
113400000000
113405000000
113410000000
113415000000
113420000000
113425000000
113430000000
113435000000
113440000000
113445000000
113450000000
113455000000
113500000000
       date         equal         first       optimal          time        0
0  20190624  2.446319e+08  2.436011e+08  2.446321e+08  110000000000   168.86
1  20190624  2.

['SBER', '20190624', '115500000000', '1', '5.csv']
115505000000
115510000000
115515000000
115520000000
115525000000
115530000000
115535000000
115540000000
115545000000
115550000000
115555000000
115600000000
115605000000
115610000000
115615000000
115620000000
115625000000
115630000000
115635000000
115640000000
115645000000
115650000000
115655000000
115700000000
115705000000
115710000000
115715000000
115720000000
115725000000
115730000000
115735000000
115740000000
115745000000
115750000000
115755000000
115800000000
115805000000
115810000000
115815000000
115820000000
115825000000
115830000000
115835000000
115840000000
115845000000
115850000000
115855000000
115900000000
115905000000
115910000000
115915000000
115920000000
115925000000
115930000000
115935000000
115940000000
115945000000
115950000000
115955000000
120000000000
        date         equal         first       optimal          time        0
0   20190624  2.446319e+08  2.436011e+08  2.446321e+08  110000000000   168.86
1   20190624 

121510000000
121515000000
121520000000
121525000000
121530000000
121535000000
121540000000
121545000000
121550000000
121555000000
121600000000
121605000000
121610000000
121615000000
121620000000
121625000000
121630000000
121635000000
121640000000
121645000000
121650000000
121655000000
121700000000
121705000000
121710000000
121715000000
121720000000
121725000000
121730000000
121735000000
121740000000
121745000000
121750000000
121755000000
121800000000
121805000000
121810000000
121815000000
121820000000
121825000000
121830000000
121835000000
121840000000
121845000000
121850000000
121855000000
121900000000
121905000000
121910000000
121915000000
121920000000
121925000000
121930000000
121935000000
121940000000
121945000000
121950000000
121955000000
122000000000
        date         equal         first       optimal          time        0
0   20190624  2.446319e+08  2.436011e+08  2.446321e+08  110000000000   168.86
1   20190624  2.446317e+08  2.436405e+08  2.446311e+08  110500000000  -659.75

...

        date         equal         first       optimal          time         0
0   20190624  2.446319e+08  2.436011e+08  2.446321e+08  110000000000    168.86
1   20190624  2.446317e+08  2.436405e+08  2.446311e+08  110500000000   -659.75
2   20190624  2.446842e+08  2.436754e+08  2.446837e+08  111000000000   -496.30
3   20190624  2.446809e+08  2.436772e+08  2.446814e+08  111500000000    539.00
4   20190624  2.446211e+08  2.436956e+08  2.446216e+08  112000000000    504.34
5   20190624  2.447814e+08  2.435792e+08  2.447783e+08  112500000000  -3077.08
6   20190624  2.450683e+08  2.437356e+08  2.450662e+08  113000000000  -2030.32
7   20190624  2.451881e+08  2.438638e+08  2.451875e+08  113500000000   -550.49
8   20190624  2.450969e+08  2.443260e+08  2.450992e+08  114000000000   2324.86
9   20190624  2.449943e+08  2.442702e+08  2.449947e+08  114500000000    411.29
10  20190624  2.451431e+08  2.438338e+08  2.451408e+08  115000000000  -2274.43
11  20190624  2.452513e+08  2.441668e+08  2.452521e+

        date         equal         first       optimal          time         0
0   20190624  2.446319e+08  2.436011e+08  2.446321e+08  110000000000    168.86
1   20190624  2.446317e+08  2.436405e+08  2.446311e+08  110500000000   -659.75
2   20190624  2.446842e+08  2.436754e+08  2.446837e+08  111000000000   -496.30
3   20190624  2.446809e+08  2.436772e+08  2.446814e+08  111500000000    539.00
4   20190624  2.446211e+08  2.436956e+08  2.446216e+08  112000000000    504.34
5   20190624  2.447814e+08  2.435792e+08  2.447783e+08  112500000000  -3077.08
6   20190624  2.450683e+08  2.437356e+08  2.450662e+08  113000000000  -2030.32
7   20190624  2.451881e+08  2.438638e+08  2.451875e+08  113500000000   -550.49
8   20190624  2.450969e+08  2.443260e+08  2.450992e+08  114000000000   2324.86
9   20190624  2.449943e+08  2.442702e+08  2.449947e+08  114500000000    411.29
10  20190624  2.451431e+08  2.438338e+08  2.451408e+08  115000000000  -2274.43
11  20190624  2.452513e+08  2.441668e+08  2.452521e+

        date         equal         first       optimal          time         0
0   20190624  2.446319e+08  2.436011e+08  2.446321e+08  110000000000    168.86
1   20190624  2.446317e+08  2.436405e+08  2.446311e+08  110500000000   -659.75
2   20190624  2.446842e+08  2.436754e+08  2.446837e+08  111000000000   -496.30
3   20190624  2.446809e+08  2.436772e+08  2.446814e+08  111500000000    539.00
4   20190624  2.446211e+08  2.436956e+08  2.446216e+08  112000000000    504.34
5   20190624  2.447814e+08  2.435792e+08  2.447783e+08  112500000000  -3077.08
6   20190624  2.450683e+08  2.437356e+08  2.450662e+08  113000000000  -2030.32
7   20190624  2.451881e+08  2.438638e+08  2.451875e+08  113500000000   -550.49
8   20190624  2.450969e+08  2.443260e+08  2.450992e+08  114000000000   2324.86
9   20190624  2.449943e+08  2.442702e+08  2.449947e+08  114500000000    411.29
10  20190624  2.451431e+08  2.438338e+08  2.451408e+08  115000000000  -2274.43
11  20190624  2.452513e+08  2.441668e+08  2.452521e+

        date         equal         first       optimal          time         0
0   20190624  2.446319e+08  2.436011e+08  2.446321e+08  110000000000    168.86
1   20190624  2.446317e+08  2.436405e+08  2.446311e+08  110500000000   -659.75
2   20190624  2.446842e+08  2.436754e+08  2.446837e+08  111000000000   -496.30
3   20190624  2.446809e+08  2.436772e+08  2.446814e+08  111500000000    539.00
4   20190624  2.446211e+08  2.436956e+08  2.446216e+08  112000000000    504.34
5   20190624  2.447814e+08  2.435792e+08  2.447783e+08  112500000000  -3077.08
6   20190624  2.450683e+08  2.437356e+08  2.450662e+08  113000000000  -2030.32
7   20190624  2.451881e+08  2.438638e+08  2.451875e+08  113500000000   -550.49
8   20190624  2.450969e+08  2.443260e+08  2.450992e+08  114000000000   2324.86
9   20190624  2.449943e+08  2.442702e+08  2.449947e+08  114500000000    411.29
10  20190624  2.451431e+08  2.438338e+08  2.451408e+08  115000000000  -2274.43
11  20190624  2.452513e+08  2.441668e+08  2.452521e+

        date         equal         first       optimal          time         0
0   20190624  2.446319e+08  2.436011e+08  2.446321e+08  110000000000    168.86
1   20190624  2.446317e+08  2.436405e+08  2.446311e+08  110500000000   -659.75
2   20190624  2.446842e+08  2.436754e+08  2.446837e+08  111000000000   -496.30
3   20190624  2.446809e+08  2.436772e+08  2.446814e+08  111500000000    539.00
4   20190624  2.446211e+08  2.436956e+08  2.446216e+08  112000000000    504.34
5   20190624  2.447814e+08  2.435792e+08  2.447783e+08  112500000000  -3077.08
6   20190624  2.450683e+08  2.437356e+08  2.450662e+08  113000000000  -2030.32
7   20190624  2.451881e+08  2.438638e+08  2.451875e+08  113500000000   -550.49
8   20190624  2.450969e+08  2.443260e+08  2.450992e+08  114000000000   2324.86
9   20190624  2.449943e+08  2.442702e+08  2.449947e+08  114500000000    411.29
10  20190624  2.451431e+08  2.438338e+08  2.451408e+08  115000000000  -2274.43
11  20190624  2.452513e+08  2.441668e+08  2.452521e+

In [37]:
# data_diff.to_csv(os.path.join(PATH, 'results_from_check/data_diff.csv'))
data_diff

,date,equal,first,optimal,time
0,20190624,2.446319e+08,2.436011e+08,2.446321e+08,110000000000
1,20190624,2.446317e+08,2.436405e+08,2.446311e+08,110500000000
2,20190624,2.446842e+08,2.436754e+08,2.446837e+08,111000000000
3,20190624,2.446809e+08,2.436772e+08,2.446814e+08,111500000000
4,20190624,2.446211e+08,2.436956e+08,2.446216e+08,112000000000
5,20190624,2.447814e+08,2.435792e+08,2.447783e+08,112500000000
6,20190624,2.450683e+08,2.437356e+08,2.450662e+08,113000000000
7,20190624,2.451881e+08,2.438638e+08,2.451875e+08,113500000000
8,20190624,2.450969e+08,2.443260e+08,2.450992e+08,114000000000
9,20190624,2.449943e+08,2.442702e+08,2.449947e+08,114500000000


In [23]:
data03 = copy.deepcopy(data_diff)
data03 = data03.loc[data03['date'] == 20190603]

data24 = copy.deepcopy(data_diff)
data24 = data24.loc[data24['date'] == 20190624]

In [24]:
sum(data03.optimal - data03.equal > 0) / len(data03.optimal)

0.28

In [25]:
sum(data03.optimal - data03.equal) / len(data03.optimal)

-4614.520799999237

In [26]:
sum(data24.optimal - data24.equal > 0) / len(data24.optimal)

0.52

In [27]:
sum(data24.optimal - data24.equal) / len(data24.optimal)

4.491599999666214

In [28]:
np.random.seed(1234)
result = {
    'date': [],
    'time': [],
    'optimal': [],
    'equal': [],
    'first': []
}
number_iters = 20
duration_iters = 30
for date in [20190603, 20190624]:
    if date == 20190603:
        volume_paste = int(np.round(103529670.5263158 / (8.75 * (6)), 0))
    elif date == 20190624:
        volume_paste = int(np.round(103529670.5263158 / (8.75 * (6)), 0))
    else:
        raise(ValueError('Something went wrong'))
    
    
    
    for time in time_creation(start=110000000000, 
                              end=130000000000, 
                              delta=5*60):

        PARAMS_TASK = {
            'ticker': 'SBER',
            'date_of_liquidation': date,
            'time_of_liquidation': time,
            'number_of_iteration': number_iters,
            'duration_of_iteration': duration_iters,
            'lambda_param': 1,
            'volume': volume_paste,
            'return_left': True
        }
        sol = problem_solution(**PARAMS_TASK)

        strategy = sol
        
        liq_value = LiquidationValue(optimal_strategy=strategy,
                                     ticker='SBER',
                                     date_of_liquidation=date,
                                     time_of_liquidation=time, 
                                     duration_of_iteration=duration_iters)

        value = liq_value.calculate_liquidation_value()

        for key in value.keys():
            result[key].append(value[key])
        
        print(pd.concat([pd.DataFrame.from_dict(result), (pd.DataFrame.from_dict(result)['optimal'] -
                                                          pd.DataFrame.from_dict(result)['equal']) ], 
                        axis=1))
    

data_eq = pd.DataFrame.from_dict(result)

       date         equal        first      optimal          time        0
0  20190603  4.585717e+08  454768783.9  458566664.4  110000000000 -4998.47
       date         equal        first       optimal          time        0
0  20190603  4.585717e+08  454768783.9  4.585667e+08  110000000000 -4998.47
1  20190603  4.588537e+08  454745024.0  4.588520e+08  110500000000 -1706.02
       date         equal        first       optimal          time        0
0  20190603  4.585717e+08  454768783.9  4.585667e+08  110000000000 -4998.47
1  20190603  4.588537e+08  454745024.0  4.588520e+08  110500000000 -1706.02
2  20190603  4.584834e+08  454747999.4  4.584885e+08  111000000000  5099.89
       date         equal         first       optimal          time        0
0  20190603  4.585717e+08  4.547688e+08  4.585667e+08  110000000000 -4998.47
1  20190603  4.588537e+08  4.547450e+08  4.588520e+08  110500000000 -1706.02
2  20190603  4.584834e+08  4.547480e+08  4.584885e+08  111000000000  5099.89
3  2019060

        date         equal         first       optimal          time         0
0   20190603  4.585717e+08  4.547688e+08  4.585667e+08  110000000000  -4998.47
1   20190603  4.588537e+08  4.547450e+08  4.588520e+08  110500000000  -1706.02
2   20190603  4.584834e+08  4.547480e+08  4.584885e+08  111000000000   5099.89
3   20190603  4.580916e+08  4.546517e+08  4.580917e+08  111500000000     50.11
4   20190603  4.580023e+08  4.547933e+08  4.580055e+08  112000000000   3214.84
5   20190603  4.574506e+08  4.547128e+08  4.574560e+08  112500000000   5414.37
6   20190603  4.571102e+08  4.539551e+08  4.571093e+08  113000000000   -874.96
7   20190603  4.572209e+08  4.542196e+08  4.572201e+08  113500000000   -792.85
8   20190603  4.569895e+08  4.544240e+08  4.569923e+08  114000000000   2880.17
9   20190603  4.576088e+08  4.544279e+08  4.575980e+08  114500000000 -10786.42
10  20190603  4.592417e+08  4.545607e+08  4.592320e+08  115000000000  -9636.82
11  20190603  4.600843e+08  4.547474e+08  4.600807e+

        date         equal         first       optimal          time         0
0   20190603  4.585717e+08  4.547688e+08  4.585667e+08  110000000000  -4998.47
1   20190603  4.588537e+08  4.547450e+08  4.588520e+08  110500000000  -1706.02
2   20190603  4.584834e+08  4.547480e+08  4.584885e+08  111000000000   5099.89
3   20190603  4.580916e+08  4.546517e+08  4.580917e+08  111500000000     50.11
4   20190603  4.580023e+08  4.547933e+08  4.580055e+08  112000000000   3214.84
5   20190603  4.574506e+08  4.547128e+08  4.574560e+08  112500000000   5414.37
6   20190603  4.571102e+08  4.539551e+08  4.571093e+08  113000000000   -874.96
7   20190603  4.572209e+08  4.542196e+08  4.572201e+08  113500000000   -792.85
8   20190603  4.569895e+08  4.544240e+08  4.569923e+08  114000000000   2880.17
9   20190603  4.576088e+08  4.544279e+08  4.575980e+08  114500000000 -10786.42
10  20190603  4.592417e+08  4.545607e+08  4.592320e+08  115000000000  -9636.82
11  20190603  4.600843e+08  4.547474e+08  4.600807e+

        date         equal         first       optimal          time         0
0   20190603  4.585717e+08  4.547688e+08  4.585667e+08  110000000000  -4998.47
1   20190603  4.588537e+08  4.547450e+08  4.588520e+08  110500000000  -1706.02
2   20190603  4.584834e+08  4.547480e+08  4.584885e+08  111000000000   5099.89
3   20190603  4.580916e+08  4.546517e+08  4.580917e+08  111500000000     50.11
4   20190603  4.580023e+08  4.547933e+08  4.580055e+08  112000000000   3214.84
5   20190603  4.574506e+08  4.547128e+08  4.574560e+08  112500000000   5414.37
6   20190603  4.571102e+08  4.539551e+08  4.571093e+08  113000000000   -874.96
7   20190603  4.572209e+08  4.542196e+08  4.572201e+08  113500000000   -792.85
8   20190603  4.569895e+08  4.544240e+08  4.569923e+08  114000000000   2880.17
9   20190603  4.576088e+08  4.544279e+08  4.575980e+08  114500000000 -10786.42
10  20190603  4.592417e+08  4.545607e+08  4.592320e+08  115000000000  -9636.82
11  20190603  4.600843e+08  4.547474e+08  4.600807e+

        date         equal         first       optimal          time         0
0   20190603  4.585717e+08  4.547688e+08  4.585667e+08  110000000000  -4998.47
1   20190603  4.588537e+08  4.547450e+08  4.588520e+08  110500000000  -1706.02
2   20190603  4.584834e+08  4.547480e+08  4.584885e+08  111000000000   5099.89
3   20190603  4.580916e+08  4.546517e+08  4.580917e+08  111500000000     50.11
4   20190603  4.580023e+08  4.547933e+08  4.580055e+08  112000000000   3214.84
5   20190603  4.574506e+08  4.547128e+08  4.574560e+08  112500000000   5414.37
6   20190603  4.571102e+08  4.539551e+08  4.571093e+08  113000000000   -874.96
7   20190603  4.572209e+08  4.542196e+08  4.572201e+08  113500000000   -792.85
8   20190603  4.569895e+08  4.544240e+08  4.569923e+08  114000000000   2880.17
9   20190603  4.576088e+08  4.544279e+08  4.575980e+08  114500000000 -10786.42
10  20190603  4.592417e+08  4.545607e+08  4.592320e+08  115000000000  -9636.82
11  20190603  4.600843e+08  4.547474e+08  4.600807e+

...

        date         equal         first       optimal          time         0
0   20190603  4.585717e+08  4.547688e+08  4.585667e+08  110000000000  -4998.47
1   20190603  4.588537e+08  4.547450e+08  4.588520e+08  110500000000  -1706.02
2   20190603  4.584834e+08  4.547480e+08  4.584885e+08  111000000000   5099.89
3   20190603  4.580916e+08  4.546517e+08  4.580917e+08  111500000000     50.11
4   20190603  4.580023e+08  4.547933e+08  4.580055e+08  112000000000   3214.84
5   20190603  4.574506e+08  4.547128e+08  4.574560e+08  112500000000   5414.37
6   20190603  4.571102e+08  4.539551e+08  4.571093e+08  113000000000   -874.96
7   20190603  4.572209e+08  4.542196e+08  4.572201e+08  113500000000   -792.85
8   20190603  4.569895e+08  4.544240e+08  4.569923e+08  114000000000   2880.17
9   20190603  4.576088e+08  4.544279e+08  4.575980e+08  114500000000 -10786.42
10  20190603  4.592417e+08  4.545607e+08  4.592320e+08  115000000000  -9636.82
11  20190603  4.600843e+08  4.547474e+08  4.600807e+

        date         equal         first       optimal          time         0
0   20190603  4.585717e+08  4.547688e+08  4.585667e+08  110000000000  -4998.47
1   20190603  4.588537e+08  4.547450e+08  4.588520e+08  110500000000  -1706.02
2   20190603  4.584834e+08  4.547480e+08  4.584885e+08  111000000000   5099.89
3   20190603  4.580916e+08  4.546517e+08  4.580917e+08  111500000000     50.11
4   20190603  4.580023e+08  4.547933e+08  4.580055e+08  112000000000   3214.84
5   20190603  4.574506e+08  4.547128e+08  4.574560e+08  112500000000   5414.37
6   20190603  4.571102e+08  4.539551e+08  4.571093e+08  113000000000   -874.96
7   20190603  4.572209e+08  4.542196e+08  4.572201e+08  113500000000   -792.85
8   20190603  4.569895e+08  4.544240e+08  4.569923e+08  114000000000   2880.17
9   20190603  4.576088e+08  4.544279e+08  4.575980e+08  114500000000 -10786.42
10  20190603  4.592417e+08  4.545607e+08  4.592320e+08  115000000000  -9636.82
11  20190603  4.600843e+08  4.547474e+08  4.600807e+

        date         equal         first       optimal          time         0
0   20190603  4.585717e+08  4.547688e+08  4.585667e+08  110000000000  -4998.47
1   20190603  4.588537e+08  4.547450e+08  4.588520e+08  110500000000  -1706.02
2   20190603  4.584834e+08  4.547480e+08  4.584885e+08  111000000000   5099.89
3   20190603  4.580916e+08  4.546517e+08  4.580917e+08  111500000000     50.11
4   20190603  4.580023e+08  4.547933e+08  4.580055e+08  112000000000   3214.84
5   20190603  4.574506e+08  4.547128e+08  4.574560e+08  112500000000   5414.37
6   20190603  4.571102e+08  4.539551e+08  4.571093e+08  113000000000   -874.96
7   20190603  4.572209e+08  4.542196e+08  4.572201e+08  113500000000   -792.85
8   20190603  4.569895e+08  4.544240e+08  4.569923e+08  114000000000   2880.17
9   20190603  4.576088e+08  4.544279e+08  4.575980e+08  114500000000 -10786.42
10  20190603  4.592417e+08  4.545607e+08  4.592320e+08  115000000000  -9636.82
11  20190603  4.600843e+08  4.547474e+08  4.600807e+

        date         equal         first       optimal          time         0
0   20190603  4.585717e+08  4.547688e+08  4.585667e+08  110000000000  -4998.47
1   20190603  4.588537e+08  4.547450e+08  4.588520e+08  110500000000  -1706.02
2   20190603  4.584834e+08  4.547480e+08  4.584885e+08  111000000000   5099.89
3   20190603  4.580916e+08  4.546517e+08  4.580917e+08  111500000000     50.11
4   20190603  4.580023e+08  4.547933e+08  4.580055e+08  112000000000   3214.84
5   20190603  4.574506e+08  4.547128e+08  4.574560e+08  112500000000   5414.37
6   20190603  4.571102e+08  4.539551e+08  4.571093e+08  113000000000   -874.96
7   20190603  4.572209e+08  4.542196e+08  4.572201e+08  113500000000   -792.85
8   20190603  4.569895e+08  4.544240e+08  4.569923e+08  114000000000   2880.17
9   20190603  4.576088e+08  4.544279e+08  4.575980e+08  114500000000 -10786.42
10  20190603  4.592417e+08  4.545607e+08  4.592320e+08  115000000000  -9636.82
11  20190603  4.600843e+08  4.547474e+08  4.600807e+

        date         equal         first       optimal          time         0
0   20190603  4.585717e+08  4.547688e+08  4.585667e+08  110000000000  -4998.47
1   20190603  4.588537e+08  4.547450e+08  4.588520e+08  110500000000  -1706.02
2   20190603  4.584834e+08  4.547480e+08  4.584885e+08  111000000000   5099.89
3   20190603  4.580916e+08  4.546517e+08  4.580917e+08  111500000000     50.11
4   20190603  4.580023e+08  4.547933e+08  4.580055e+08  112000000000   3214.84
5   20190603  4.574506e+08  4.547128e+08  4.574560e+08  112500000000   5414.37
6   20190603  4.571102e+08  4.539551e+08  4.571093e+08  113000000000   -874.96
7   20190603  4.572209e+08  4.542196e+08  4.572201e+08  113500000000   -792.85
8   20190603  4.569895e+08  4.544240e+08  4.569923e+08  114000000000   2880.17
9   20190603  4.576088e+08  4.544279e+08  4.575980e+08  114500000000 -10786.42
10  20190603  4.592417e+08  4.545607e+08  4.592320e+08  115000000000  -9636.82
11  20190603  4.600843e+08  4.547474e+08  4.600807e+

In [36]:
# data_eq.to_csv(os.path.join(PATH, 'results_from_check/data_eq.csv'))
data_eq

,date,equal,first,optimal,time
0,20190603,4.585717e+08,4.547688e+08,4.585667e+08,110000000000
1,20190603,4.588537e+08,4.547450e+08,4.588520e+08,110500000000
2,20190603,4.584834e+08,4.547480e+08,4.584885e+08,111000000000
3,20190603,4.580916e+08,4.546517e+08,4.580917e+08,111500000000
4,20190603,4.580023e+08,4.547933e+08,4.580055e+08,112000000000
5,20190603,4.574506e+08,4.547128e+08,4.574560e+08,112500000000
6,20190603,4.571102e+08,4.539551e+08,4.571093e+08,113000000000
7,20190603,4.572209e+08,4.542196e+08,4.572201e+08,113500000000
8,20190603,4.569895e+08,4.544240e+08,4.569923e+08,114000000000
9,20190603,4.576088e+08,4.544279e+08,4.575980e+08,114500000000


In [30]:
data03 = copy.deepcopy(data_eq)
data03 = data03.loc[data03['date'] == 20190603]

data24 = copy.deepcopy(data_eq)
data24 = data24.loc[data24['date'] == 20190624]

In [31]:
sum(data03.optimal - data03.equal > 0) / len(data03.optimal)

0.32

In [32]:
sum(data03.optimal - data03.equal) / len(data03.optimal)

-4550.212399995327

In [33]:
sum(data24.optimal - data24.equal > 0) / len(data24.optimal)

0.48

In [34]:
sum(data24.optimal - data24.equal) / len(data24.optimal)

-39.33279999256134